This notebook shows you how to create and query a table or DataFrame loaded from data stored in Azure Blob storage.

### Step 1: Set the data location and type

There are two ways to access Azure Blob storage: account keys and shared access signatures (SAS).

To get started, we need to set the location and type of the file.

In [0]:
storage_account_name = "dataingest002581"
container_name = "electricity-data"

In [0]:
scope_name = "electricity-scope"
print(dbutils.secrets.list(scope_name))

[SecretMetadata(key='storage-account-access-key'), SecretMetadata(key='storage-account-name')]

In [0]:
file_location = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net"
conf_key = f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net"
mount_point = f"/mnt/{container_name}"

if mount_point in [mnt.mountPoint for mnt in dbutils.fs.mounts()]:
  dbutils.fs.unmount(mount_point)

/mnt/electricity-data has been unmounted.

In [0]:
dbutils.fs.mount(
  source = file_location,
  mount_point = mount_point,
  extra_configs = {f"{conf_key}":dbutils.secrets.get(scope = scope_name, key = "storage-account-access-key")})

Out[46]: True

In [0]:
import os
import numpy as np
import databricks.koalas as ks

dbutils.fs.mkdirs(os.path.join(mount_point, 'bronze'))
dbutils.fs.ls(mount_point + '/bronze')
print(f"Raw Files:")
[print("Path:", f.path, "\tSize:", f.size) for f in dbutils.fs.ls(mount_point)]
print(f"\n")

--------------------------------------------------------------------------- 
 ModuleNotFoundError Traceback (most recent call last)
 <command-1121945877521148> in <module> 
 2 import numpy as np
 3 import databricks . koalas as ks
 ----> 4 import pyspark . pandas as ps
 5 
 6 dbutils . fs . mkdirs ( os . path . join ( mount_point , 'bronze' ) ) 

 /databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py in import_patch (name, globals, locals, fromlist, level) 
 156 # Import the desired module. If you’re seeing this while debugging a failed import, 
 157 # look at preceding stack frames for relevant error information. 
 --> 158 original_result = python_builtin_import ( name , globals , locals , fromlist , level ) 
 159 
 160 is_root_import = thread_local . _nest_level == 1 

 ModuleNotFoundError : No module named 'pyspark.pandas'

### Step 2: Read the data

Now that we have specified our file metadata, we can create a DataFrame. Notice that we use an *option* to specify that we want to infer the schema from the file. We can also explicitly set this to a particular schema if we have one already.

First, let's create a DataFrame in Python.

In [0]:
file_list = dbutils.fs.ls(mount_point)
# file_location = file_list[2].path
file_location = "dbfs:/mnt/electricity-data/f923_2011.csv"
file_type = "csv"
write_location = file_list[0].path + file_location.split('/')[-1].split('.')[0]
print(f"Read Location: {file_location}"); print(f"Write Location: {write_location}")

Read Location: dbfs:/mnt/electricity-data/f923_2011.csv
Write Location: dbfs:/mnt/electricity-data/bronze/f923_2011

In [0]:
df = ks.read_csv("dbfs:/mnt/electricity-data/f923_2011.csv", header=0) # file_location, header=0)
df.columns = [s.replace(" ", "_").replace("&", "and").replace(".", "_") for s in df.columns]
df.columns = [s.replace(r"(", r"").replace(r")", r"") for s in df.columns]
ddl = [df.spark.schema().simpleString().split("<")[1].split(">")[0]]
ddl = [s.replace(':', ' ') for s in ddl][0]

In [0]:
dbutils.fs.ls(mount_point + "/bronze")

Out[198]: [FileInfo(path='dbfs:/mnt/electricity-data/bronze/f923_2008/', name='f923_2008/', size=0),
 FileInfo(path='dbfs:/mnt/electricity-data/bronze/f923_2009/', name='f923_2009/', size=0),
 FileInfo(path='dbfs:/mnt/electricity-data/bronze/f923_2010/', name='f923_2010/', size=0),
 FileInfo(path='dbfs:/mnt/electricity-data/bronze/f923_2011/', name='f923_2011/', size=0),
 FileInfo(path='dbfs:/mnt/electricity-data/bronze/f923_2012/', name='f923_2012/', size=0),
 FileInfo(path='dbfs:/mnt/electricity-data/bronze/f923_2013/', name='f923_2013/', size=0),
 FileInfo(path='dbfs:/mnt/electricity-data/bronze/f923_2014/', name='f923_2014/', size=0),
 FileInfo(path='dbfs:/mnt/electricity-data/bronze/f923_2015/', name='f923_2015/', size=0),
 FileInfo(path='dbfs:/mnt/electricity-data/bronze/f923_2016/', name='f923_2016/', size=0),
 FileInfo(path='dbfs:/mnt/electricity-data/bronze/f923_2017/', name='f923_2017/', size=0),
 FileInfo(path='dbfs:/mnt/electricity-data/bronze/f923_2018/', name='f923_2018/', size=0),
 FileInfo(path='dbfs:/mnt/electricity-data/bronze/f923_2019/', name='f923_2019/', size=0),
 FileInfo(path='dbfs:/mnt/electricity-data/bronze/f923_2020/', name='f923_2020/', size=0),
 FileInfo(path='dbfs:/mnt/electricity-data/bronze/f923_2021/', name='f923_2021/', size=0)]

In [0]:
for path in [f.path for f in dbutils.fs.ls(mount_point) if f.size > 0]:
  name = path.split('/')[-1].split('.')[0]
  write_location = f"dbfs:/mnt/electricity-data/bronze/{name}"
  df = ks.read_csv(path, header=None, names=ddl) # if df.columns[-1].lower() != 'year' else ks.read_csv(path, header=0)
  print(f"header length {name}: {df.columns.size}")
  assert df.columns[-1].lower() == 'year', f"{name} last field should be 'year'"
  try:
    df.to_delta(write_location)
  except Exception as err:
    print(f"{err} in {path.split(r'/')[-1]}")

header length f923_2008: 97
header length f923_2009: 97
header length f923_2010: 97
header length f923_2011: 97
header length f923_2012: 97
header length f923_2013: 97
header length f923_2014: 97
header length f923_2015: 97
header length f923_2016: 97
header length f923_2017: 97
header length f923_2018: 97
header length f923_2019: 97
header length f923_2020: 97
header length f923_2021: 97

In [0]:
dbutils.fs.mkdirs(os.path.join(mount_point, 'silver'))
dbutils.fs.mkdirs(os.path.join(mount_point, 'gold'))
for f in dbutils.fs.ls(mount_point):
  if f.size <=0:
    print(f.path)
# [print(f.path) for f in dbutils.fs.ls(os.path.join(mount_point, 'bronze', 'f923_2020'))]

dbfs:/mnt/electricity-data/bronze/
dbfs:/mnt/electricity-data/gold/
dbfs:/mnt/electricity-data/silver/

In [0]:
silver_path = os.path.join(mount_point, 'silver')

In [0]:
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', 'true')
# df = spark.read.format("delta").load(os.path.join(mount_point, 'bronze', 'f923_2008')).to_koalas()
df = ks.read_delta(os.path.join(mount_point, 'bronze', 'f923_2008'))
df = df.loc[df['Plant_Id'].notnull(), :]
df = df.drop(['Operator_Id'], axis=1)
df = df.drop(df.columns[df.columns.str.startswith("Reserved")], axis=1)
df = df.drop(df.columns[df.columns.str.startswith("Netgen_")], axis=1)
df = df.drop(df.columns[df.columns.str.startswith("Elec_MMBtu")], axis=1)
df = df.drop(df.columns[df.columns.str.startswith("Tot_MMBtu")], axis=1)
df = df.drop(df.columns[df.columns.str.startswith("MMBtuPer_Unit_")], axis=1)
df = df.drop(df.columns[df.columns.str.startswith("Elec_Quantity_")], axis=1)
df = df.drop(df.columns[df.columns.str.startswith("Quantity_")], axis=1)

In [0]:
ks.set_option('compute.ops_on_diff_frames', True)
df[['Plant_Id', 'Nuclear_Unit_Id', 'NAICS_Code', 'EIA_Sector_Number', 'YEAR']] = df[['Plant_Id', 'Nuclear_Unit_Id', 'NAICS_Code', 'EIA_Sector_Number', 'YEAR']].apply(lambda _: _.astype(np.float).astype("Int32"))
ks.set_option('compute.ops_on_diff_frames', False)

In [0]:
write_path = mount_point + '/silver/f923_ytd_2008'
# df.to_delta(write_path, mode='overwrite')
df.to_spark_io(write_path, format='delta', mode='overwrite')

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-4173014708317328> in <module> 
 1 write_path = mount_point + '/silver/f923_ytd_2008' 
 2 # df.to_delta(write_path, mode='overwrite') 
 ----> 3 df . to_spark_io ( write_path , format = 'delta' , mode = 'overwrite' ) 

 /databricks/python/lib/python3.8/site-packages/databricks/koalas/usage_logging/__init__.py in wrapper (*args, **kwargs) 
 193 start = time . perf_counter ( ) 
 194 try : 
 --> 195 res = func ( * args , ** kwargs ) 
 196 logger.log_success(
 197 class_name , function_name , time . perf_counter ( ) - start , signature

 /databricks/python/lib/python3.8/site-packages/databricks/koalas/frame.py in to_spark_io (self, path, format, mode, partition_cols, index_col, **options) 
 4868 ** options
 4869 ) -> None:
 -> 4870 return self . spark . to_spark_io ( path , format , mode , partition_cols , index_col , ** options ) 
 4871 
 4872 to_spark_io . __doc__ = SparkFrameMethods . to_spark_io . __doc__

 /databricks/python/lib/python3.8/site-packages/databricks/koalas/usage_logging/__init__.py in wrapper (*args, **kwargs) 
 188 if hasattr ( _local , "logging" ) and _local . logging : 
 189 # no need to log since this should be internal call. 
 --> 190 return func ( * args , ** kwargs ) 
 191 _local . logging = True 
 192 try : 

 /databricks/python/lib/python3.8/site-packages/databricks/koalas/spark/accessors.py in to_spark_io (self, path, format, mode, partition_cols, index_col, **options) 
 824 options = options . get ( "options" ) # type: ignore 
 825 
 --> 826 self._kdf.spark.frame(index_col=index_col).write.save(
 827 path = path , format = format , mode = mode , partitionBy = partition_cols , ** options
 828 )

 /databricks/spark/python/pyspark/sql/readwriter.py in save (self, path, format, mode, partitionBy, **options) 
 1134 self . _jwrite . save ( ) 
 1135 else : 
 -> 1136 self . _jwrite . save ( path ) 
 1137 
 1138 @ since ( 1.4 ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o673357.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:307)
	at com.databricks.sql.transaction.tahoe.files.TransactionalWriteEdge.$anonfun$writeFiles$5(TransactionalWriteEdge.scala:349)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$5(SQLExecution.scala:130)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:273)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$1(SQLExecution.scala:104)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:854)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:77)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:223)
	at com.databricks.sql.transaction.tahoe.files.TransactionalWriteEdge.$anonfun$writeFiles$

In [0]:
df = df.loc[df['Plant_Id'].notnull(), ]
df['Plant_Id'] = df['Plant_Id'].map(int)

In [0]:
tbl = spark.read.format("delta").load(os.path.join(mount_point, 'bronze', 'f923_2009'))
display(tbl)

Plant_Id,Combined_Heat_and_Power_Plant,Nuclear_Unit_Id,Plant_Name,Operator_Name,Operator_Id,State,Census_Region,NERC_Region,Reserved,NAICS_Code,EIA_Sector_Number,Sector_Name,Reported_Prime_Mover,Reported_Fuel_Type_Code,AER_Fuel_Type_Code,Reserved_1,Reserved_2,Physical_Unit_Label,Quantity_Jan,Quantity_Feb,Quantity_Mar,Quantity_Apr,Quantity_May,Quantity_Jun,Quantity_Jul,Quantity_Aug,Quantity_Sep,Quantity_Oct,Quantity_Nov,Quantity_Dec,Elec_Quantity_Jan,Elec_Quantity_Feb,Elec_Quantity_Mar,Elec_Quantity_Apr,Elec_Quantity_May,Elec_Quantity_Jun,Elec_Quantity_Jul,Elec_Quantity_Aug,Elec_Quantity_Sep,Elec_Quantity_Oct,Elec_Quantity_Nov,Elec_Quantity_Dec,MMBtuPer_Unit_Jan,MMBtuPer_Unit_Feb,MMBtuPer_Unit_Mar,MMBtuPer_Unit_Apr,MMBtuPer_Unit_May,MMBtuPer_Unit_Jun,MMBtuPer_Unit_Jul,MMBtuPer_Unit_Aug,MMBtuPer_Unit_Sep,MMBtuPer_Unit_Oct,MMBtuPer_Unit_Nov,MMBtuPer_Unit_Dec,Tot_MMBtuJan,Tot_MMBtuFeb,Tot_MMBtuMar,Tot_MMBtuApr,Tot_MMBtuMay,Tot_MMBtuJun,Tot_MMBtuJul,Tot_MMBtuAug,Tot_MMBtuSep,Tot_MMBtuOct,Tot_MMBtuNov,Tot_MMBtuDec,Elec_MMBtuJan,Elec_MMBtuFeb,Elec_MMBtuMar,Elec_MMBtuApr,Elec_MMBtuMay,Elec_MMBtuJun,Elec_MMBtuJul,Elec_MMBtuAug,Elec_MMBtuSep,Elec_MMBtuOct,Elec_MMBtuNov,Elec_MMBtuDec,Netgen_Jan,Netgen_Feb,Netgen_Mar,Netgen_Apr,Netgen_May,Netgen_Jun,Netgen_Jul,Netgen_Aug,Netgen_Sep,Netgen_Oct,Netgen_Nov,Netgen_Dec,Total_Fuel_Consumption_Quantity,Electric_Fuel_Consumption_Quantity,Total_Fuel_Consumption_MMBtu,Elec_Fuel_Consumption_MMBtu,Net_Generation_Megawatthours,YEAR
null,Combined Heat & Power Plant,null,Plant Name,Operator Name,null,State,Census Region,NERC Region,Reserved,null,null,Sector Name,Reported Prime Mover,Reported Fuel Type Code,AER Fuel Type Code,Reserved .1,Reserved .2,Physical Unit Label,QUANTITY_JAN,QUANTITY_FEB,QUANTITY_MAR,QUANTITY_APR,QUANTITY_MAY,QUANTITY_JUN,QUANTITY_JUL,QUANTITY_AUG,QUANTITY_SEP,QUANTITY_OCT,QUANTITY_NOV,QUANTITY_DEC,ELEC_QUANTITY_JAN,ELEC_QUANTITY_FEB,ELEC_QUANTITY_MAR,ELEC_QUANTITY_APR,ELEC_QUANTITY_MAY,ELEC_QUANTITY_JUN,ELEC_QUANTITY_JUL,ELEC_QUANTITY_AUG,ELEC_QUANTITY_SEP,ELEC_QUANTITY_OCT,ELEC_QUANTITY_NOV,ELEC_QUANTITY_DEC,MMBTU_PER_UNIT_JAN,MMBTU_PER_UNIT_FEB,MMBTU_PER_UNIT_MAR,MMBTU_PER_UNIT_APR,MMBTU_PER_UNIT_MAY,MMBTU_PER_UNIT_JUN,MMBTU_PER_UNIT_JUL,MMBTU_PER_UNIT_AUG,MMBTU_PER_UNIT_SEP,MMBTU_PER_UNIT_OCT,MMBTU_PER_UNIT_NOV,MMBTU_PER_UNIT_DEC,TOT_MMBTU_JAN,TOT_MMBTU_FEB,TOT_MMBTU_MAR,TOT_MMBTU_APR,TOT_MMBTU_MAY,TOT_MMBTU_JUN,TOT_MMBTU_JUL,TOT_MMBTU_AUG,TOT_MMBTU_SEP,TOT_MMBTU_OCT,TOT_MMBTU_NOV,TOT_MMBTU_DEC,ELEC_MMBTUS_JAN,ELEC_MMBTUS_FEB,ELEC_MMBTUS_MAR,ELEC_MMBTUS_APR,ELEC_MMBTUS_MAY,ELEC_MMBTUS_JUN,ELEC_MMBTUS_JUL,ELEC_MMBTUS_AUG,ELEC_MMBTUS_SEP,ELEC_MMBTUS_OCT,ELEC_MMBTUS_NOV,ELEC_MMBTUS_DEC,NETGEN_JAN,NETGEN_FEB,NETGEN_MAR,NETGEN_APR,NETGEN_MAY,NETGEN_JUN,NETGEN_JUL,NETGEN_AUG,NETGEN_SEP,NETGEN_OCT,NETGEN_NOV,NETGEN_DEC,null,null,null,null,null,null
2,N,null,Bankhead Dam,Alabama Power Co,195,AL,ESC,SERC,null,22,1,Electric Utility,HY,WAT,HYC,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,282335.0,119344.0,256672.0,220520.0,286025.0,116444.0,83031.0,110115.0,202894.0,333608.0,328891.0,418871.0,282335.0,119344.0,256672.0,220520.0,286025.0,116444.0,83031.0,110115.0,202894.0,333608.0,328891.0,418871.0,28927.787,12227.884,26298.379,22594.291,29305.852,11930.788,8507.325,11282.228,20788.348,34181.161,33697.834,42917.123,0,0,null,null,282659.0,2009
3,N,null,Barry,Alabama Power Co,195,AL,ESC,SERC,null,22,1,Electric Utility,CA,NG,NG,null,null,mcf,0.0,0.0,4002.0,40457.0,41355.0,66851.0,63820.0,36144.0,24808.0,12874.0,3118.0,11078.0,0.0,0.0,4002.0,40457.0,41355.0,66851.0,63820.0,36144.0,24808.0,12874.0,3118.0,11078.0,0.0,0.0,1.03,1.032,1.03,1.028,1.0310000000000001,1.03,1.025,1.025,1.035,1.0290000000000001,0.0,0.0,4122.0,41752.0,42596.0,68723.0,65798.0,37228.0,25428.0,13196.0,3227.0,11399.0,0.0,0.0,4122.0,41752.0,42596.0,68723.0,65798.0,37228.0,25428.0,13196.0,3227.0,11399.0,96886.0,113

In [0]:
spark.sql("""CREATE DATABASE f923""")

Out[224]: DataFrame[]

In [0]:
spark.sql("""CREATE TABLE IF NOT EXISTS 'f923.generation_and_fuel_data' USING DELTA PARTITIONED BY (year) LOCATION 'dbfs:/mnt/electricity-data/silver/generation_and_fuel_data'""")

In [0]:
df.columns

Out[201]: ['Plant_Id',
 'Combined_Heat_and_Power_Plant',
 'Nuclear_Unit_Id',
 'Plant_Name',
 'Operator_Name',
 'Operator_Id',
 'State',
 'Census_Region',
 'NERC_Region',
 'Reserved',
 'NAICS_Code',
 'EIA_Sector_Number',
 'Sector_Name',
 'Reported_Prime_Mover',
 'Reported_Fuel_Type_Code',
 'AER_Fuel_Type_Code',
 'Reserved_1',
 'Reserved_2',
 'Physical_Unit_Label',
 'Quantity_Jan',
 'Quantity_Feb',
 'Quantity_Mar',
 'Quantity_Apr',
 'Quantity_May',
 'Quantity_Jun',
 'Quantity_Jul',
 'Quantity_Aug',
 'Quantity_Sep',
 'Quantity_Oct',
 'Quantity_Nov',
 'Quantity_Dec',
 'Elec_Quantity_Jan',
 'Elec_Quantity_Feb',
 'Elec_Quantity_Mar',
 'Elec_Quantity_Apr',
 'Elec_Quantity_May',
 'Elec_Quantity_Jun',
 'Elec_Quantity_Jul',
 'Elec_Quantity_Aug',
 'Elec_Quantity_Sep',
 'Elec_Quantity_Oct',
 'Elec_Quantity_Nov',
 'Elec_Quantity_Dec',
 'MMBtuPer_Unit_Jan',
 'MMBtuPer_Unit_Feb',
 'MMBtuPer_Unit_Mar',
 'MMBtuPer_Unit_Apr',
 'MMBtuPer_Unit_May',
 'MMBtuPer_Unit_Jun',
 'MMBtuPer_Unit_Jul',
 'MMBtuPer_Unit_Aug',
 'MMBtuPer_Unit_Sep',
 'MMBtuPer_Unit_Oct',
 'MMBtuPer_Unit_Nov',
 'MMBtuPer_Unit_Dec',
 'Tot_MMBtuJan',
 'Tot_MMBtuFeb',
 'Tot_MMBtuMar',
 'Tot_MMBtuApr',
 'Tot_MMBtuMay',
 'Tot_MMBtuJun',
 'Tot_MMBtuJul',
 'Tot_MMBtuAug',
 'Tot_MMBtuSep',
 'Tot_MMBtuOct',
 'Tot_MMBtuNov',
 'Tot_MMBtuDec',
 'Elec_MMBtuJan',
 'Elec_MMBtuFeb',
 'Elec_MMBtuMar',
 'Elec_MMBtuApr',
 'Elec_MMBtuMay',
 'Elec_MMBtuJun',
 'Elec_MMBtuJul',
 'Elec_MMBtuAug',
 'Elec_MMBtuSep',
 'Elec_MMBtuOct',
 'Elec_MMBtuNov',
 'Elec_MMBtuDec',
 'Netgen_Jan',
 'Netgen_Feb',
 'Netgen_Mar',
 'Netgen_Apr',
 'Netgen_May',
 'Netgen_Jun',
 'Netgen_Jul',
 'Netgen_Aug',
 'Netgen_Sep',
 'Netgen_Oct',
 'Netgen_Nov',
 'Netgen_Dec',
 'Total_Fuel_Consumption_Quantity',
 'Electric_Fuel_Consumption_Quantity',
 'Total_Fuel_Consumption_MMBtu',
 'Elec_Fuel_Consumption_MMBtu',
 'Net_Generation_Megawatthours',
 'YEAR']

### Step 3: Query the data

Now that we have created our DataFrame, we can query it. For instance, you can identify particular columns to select and display.

In [0]:
display(df.select("EXAMPLE_COLUMN"))

### Step 4: (Optional) Create a view or table

If you want to query this data as a table, you can simply register it as a *view* or a table.

In [0]:
df.createOrReplaceTempView("YOUR_TEMP_VIEW_NAME")

We can query this view using Spark SQL. For instance, we can perform a simple aggregation. Notice how we can use `%sql` to query the view from SQL.

In [0]:
%sql

SELECT EXAMPLE_GROUP, SUM(EXAMPLE_AGG) FROM YOUR_TEMP_VIEW_NAME GROUP BY EXAMPLE_GROUP

Since this table is registered as a temp view, it will be available only to this notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.

In [0]:
df.write.format("parquet").saveAsTable("MY_PERMANENT_TABLE_NAME")

This table will persist across cluster restarts and allow various users across different notebooks to query this data.